In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import KDTree
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm
import torch
import torch.nn as nn
import copy
import os
from sklearn.metrics import f1_score,recall_score, accuracy_score, roc_auc_score,cohen_kappa_score,confusion_matrix,precision_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
import random

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class Incremental_sampling2(object):
    def __init__(self,save_prototype_nums = 30):
        self.save_prototype_nums = save_prototype_nums
        self.incremental_prototypes = {}
        self.data  = np.array([])
        self.label = np.array([])
        self.combined_map = {}
    def split_current_data_by_class(self):
        """把当前轮的数据按类别划分"""
        # 获取唯一的类别标签
        current_unique_labels = np.unique(self.label)
        # 按类别分割当前轮获得的数据
        data_by_class = {}  # map {"label":[the data belong to the label]}
        for current_data_label in current_unique_labels:
            indices = np.where(self.label == current_data_label)[0]
            data_by_class[current_data_label] = self.data[indices]
        return data_by_class, current_unique_labels
    def data_combined(self):
        """
        上一轮获得的数据原型和这一轮的新数据进行合并
        1.首先，按不同类别把数据和原型进行分开
        2.判断是不是新出现的类别的数据
            2.1、新类别数据,在原型map中直接扩展一个新类的map,{'新类':新类的数据}
            2.2、原先类别的数据,在对应类的数据上进行扩展，{'已有类':已有数据+新数据}
        """
        current_data_map_by_class, current_unique_labels = self.split_current_data_by_class()
        for new_data_label in current_unique_labels:
            if new_data_label in self.incremental_prototypes:
                # 增量map中已经有这个标签的数据了，那就扩充这个数据
                self.incremental_prototypes[new_data_label] = np.concatenate(
                    (self.incremental_prototypes[new_data_label], current_data_map_by_class[new_data_label])
                )
            else:
                # 如果增量map中没有这个标签的的数据，就扩充增量map
                self.incremental_prototypes[new_data_label] = current_data_map_by_class[new_data_label]
        self.combined_map = self.incremental_prototypes
    def cut_down_nearest_data_eu(self):
        """
        把每个类对应的原型中减去超出save_prototype_nums的样本
        判断规则：
            从离得最近的开始删除
        """
        for label in self.incremental_prototypes:
            data = self.incremental_prototypes[label]
            if len(data) > self.save_prototype_nums:
                # 计算成对距离
                pairwise_distances = squareform(pdist(data, 'euclidean'))
                np.fill_diagonal(pairwise_distances, np.inf)  # 将自身距离设置为无穷大，忽略自身距离

                while len(data) > self.save_prototype_nums:
                    # 找到距离最小的一对
                    min_dist_indices = np.unravel_index(np.argmin(pairwise_distances), pairwise_distances.shape)
                    # 保留一个样本，删除另一个
                    data = np.delete(data, min_dist_indices[1], axis=0)
                    # 从距离矩阵中删除对应的行和列
                    pairwise_distances = np.delete(pairwise_distances, min_dist_indices[1], axis=0)
                    pairwise_distances = np.delete(pairwise_distances, min_dist_indices[1], axis=1)

                # 使用简化后的数据更新原型
                self.incremental_prototypes[label] = data
    def cut_down_nearest_data(self):
        """
        把每个类对应的原型中减去超出save_prototype_nums的样本
        判断规则：
            从离得最近的开始删除
        """
        for label in self.incremental_prototypes:
            # self.incremental_prototypes 在这之前已经和新数据合并了
            data = self.incremental_prototypes[label]
            if len(data) > self.save_prototype_nums:
                while len(data) > self.save_prototype_nums:
                    # 建立 KD 树
                    kdtree = KDTree(data)
                    # 查询每个点的最近邻
                    distances, indices = kdtree.query(data, k=2)  # k=2 因为第一个最近邻是点本身

                    # 找到最近的两个点
                    min_dist_idx = np.argmin(distances[:, 1])  # distances[:, 1] 是每个点的最近邻距离
                    nearest_idx = indices[min_dist_idx, 1]

                    # 删除其中一个点
                    data = np.delete(data, nearest_idx, axis=0)

                # 使用简化后的数据更新原型
                self.incremental_prototypes[label] = data
    def compute_sampling_nums(self,combined_map):
        # initialize
        min_length = float('inf')  # 初始化最小长度为正无穷大
        max_length = 0  # 初始化最大长度为0
        for label, data in combined_map.items():
            data_length = len(data)
            if data_length < min_length:
                min_length = data_length
            if data_length > max_length:
                max_length = data_length
        return min_length, max_length
    def triplet_sampling(self, num_cluster, n_neighbors=5, randomOr=True, len_lim=True):
        """Triplets 数据采样"""
        gen_x = []
        gen_y = []
        for label in self.incremental_prototypes:
            data_min = self.incremental_prototypes[label]
            if len(data_min) < num_cluster:
                size = num_cluster - len(data_min)
                weight = np.ones(len(data_min))
                # 收集多数类样本
                data_maj = np.vstack([self.incremental_prototypes[l] for l in self.incremental_prototypes if l != label])
                gen_x_c, gen_y_c = self._sample_one(data_min, data_maj, label, size, weight, n_neighbors, randomOr, len_lim)
                gen_x += gen_x_c
                gen_y += gen_y_c
        resampling_data = []
        resampling_label = []
        for label in self.combined_map: # incremental_prototypes样本太少了，只是为了生成样本用的
            data = self.incremental_prototypes[label]
            resampling_data.append(data)
            resampling_label.extend([label] * len(data))
        resampling_data = np.vstack(resampling_data)
        resampling_label = np.array(resampling_label)
        if len(gen_x) > 0:
            gen_x = np.vstack(gen_x)
            gen_y = np.array(gen_y)
            resampling_data = np.concatenate((resampling_data, gen_x), axis=0)
            resampling_label = np.concatenate((resampling_label, gen_y), axis=0)
        # 洗牌
        combined_data = list(zip(resampling_data, resampling_label))
        random.shuffle(combined_data)
        resampling_data, resampling_label = zip(*combined_data)
        return resampling_data,resampling_label
    def _sample_one(self, data_min, data_maj, label, size, weight, n_neighbors, randomOr, len_lim):
        gen_x = []
        gen_y = []
        if size == 0: return gen_x, gen_y

        min_idxs = np.arange(len(data_min))
        nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(data_maj)
        _, indices = nbrs.kneighbors(data_min)

        for j in np.random.choice(len(min_idxs), size, p=weight / weight.sum()):
            tp1 = data_min[min_idxs[j]]
            tp2 = data_maj[indices[j][:5]].mean(axis=0)
            tp3_ord = np.random.randint(n_neighbors)
            tp3 = data_maj[indices[j][tp3_ord]]
            if (tp2 == tp1).all():
                gen_x.append(tp1)
                gen_y.append(label)
                continue

            offset = tp3 - tp2
            offset_norm = norm(offset)
            if offset_norm == 0:
                continue

            tp1_tp2_norm = norm(tp1 - tp2)
            if tp1_tp2_norm == 0:
                continue

            if len_lim: offset = offset * min(1, tp1_tp2_norm / offset_norm)
            coef = np.random.rand() if randomOr else 1.0
            new_x = tp1 + coef * offset
            gen_x.append(new_x)
            gen_y.append(label)

        return gen_x, gen_y

    def random_sampling(self,num_cluster,sampling_strategy):
        """
        这里需要对数据采样
        首先遍历self.incremental_prototypes ,每个类
        以及每个类的数据的长度
        然后比较每个类的数据的长度和num_cluster之间的差距
        差额部分使用triplets的核心算法对齐进行生成样本
        """
        resampling_data = []
        resampling_label = []
        if sampling_strategy == "ros-p":
            # 随机上采样，差额数据从原型数据中随机复制
            for label, data in self.combined_map.items():
                if len(data) < num_cluster:  # 需要上采样的数据的条件
                    prototype_data = self.incremental_prototypes[label]
                    sampling_nums = num_cluster - len(data)

                    if len(prototype_data) < sampling_nums:
                        sampled_data = []
                        while len(sampled_data) < sampling_nums:
                            # 原型中样本还没有sampling_nums多时，直接复制原型中的数据
                            needed = sampling_nums - len(sampled_data)
                            sampled_indices = random.sample(range(len(prototype_data)), min(needed, len(prototype_data)))
                            sampled_data.extend([prototype_data[i] for i in sampled_indices])
                    else:
                        sampled_indices = random.sample(range(len(prototype_data)), sampling_nums)
                        sampled_data = [prototype_data[i] for i in sampled_indices]

                    resampling_data.extend(data)
                    resampling_label.extend([label] * len(data))
                    resampling_data.extend(sampled_data)
                    resampling_label.extend([label] * sampling_nums)
                elif len(data) == num_cluster:
                    resampling_data.extend(data)
                    resampling_label.extend([label] * len(data))
        elif sampling_strategy == "ros-h" :
            # 随机上采样，从混合数据中采样差额数据
            for label, data in self.combined_map.items():
                data_len = len(data)
                if data_len < num_cluster:
                    # 那就是下采样了,不放回采样
                    sampling_nums = num_cluster-data_len
                    if data_len < sampling_nums:
                        sampled_data = []
                        while len(sampled_data) < sampling_nums:
                            # 原型中样本还没有sampling_nums多时，直接复制原型中的数据
                            needed = sampling_nums - len(sampled_data)
                            sampled_indices = random.sample(range(data_len), min(needed, data_len))
                            sampled_data.extend([data[i] for i in sampled_indices])
                    else:
                        sampled_indices = random.sample(range(data_len), sampling_nums)
                        sampled_data = [data[i] for i in sampled_indices]
                    resampling_data.extend(data)
                    resampling_label.extend([label] * len(data))
                    resampling_data.extend(sampled_data)
                    resampling_label.extend([label] * sampling_nums)
                elif len(data) == num_cluster:
                    # 直接复制
                    resampling_data.extend(data)
                    resampling_label.extend([label] * len(data))
        # 洗牌
        combined_data = list(zip(resampling_data, resampling_label))
        random.shuffle(combined_data)
        resampling_data, resampling_label = zip(*combined_data)
        return resampling_data,resampling_label
    def fit(self,new_data,new_data_label,last_round_prototype,sampling_strategy='tpl'):
        self.incremental_prototypes = last_round_prototype
        self.data = new_data
        self.label = new_data_label
        self.data_combined()
        resampling_data = []
        resampling_label = []
        min_length, max_length =self.compute_sampling_nums(self.incremental_prototypes)
        if sampling_strategy.lower() == "tpl":
            resampling_data,resampling_label = self.triplet_sampling(max_length)
        elif sampling_strategy.lower() == "ros-p":
            # rest data copied from prototype
            resampling_data,resampling_label = self.random_sampling(max_length,sampling_strategy = "ros-p")
            print('use ros p')
        elif sampling_strategy.lower() == "ros-h":
            # rest data copied from hybrid data(combined data)
            resampling_data,resampling_label = self.random_sampling(max_length,sampling_strategy = "ros-h")
            print('use ros h')
        return resampling_data,resampling_label,self.incremental_prototypes

def train_model_FedAvg_local(input_model,X_train_tensor, y_train_tensor, num_epochs):
    losses =[]
    model = input_model# copy.deepcopy(input_model)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        losses.append(loss.item())  # 将每次训练的损失值添加到列表中

    return losses
def train_model_FedProx_local(input_model, X_train_tensor, y_train_tensor, num_epochs):
    mu = 0.1
    # because last round trained global model replaced local model,
    # that in this round the first local model is last round global model
    losses =[]
    model = input_model# copy.deepcopy(input_model)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    global_weights = copy.deepcopy(list(model.parameters()))
    # not use deepcopy ,because model as parameter transport in this ,update model also update model
    # current_local_model = cmodel
    # model.train()
    # for epoch in range(num_epochs):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        # FedProx
        prox_term = 0.0
        for p_i, param in enumerate(model.parameters()):
                prox_term += (mu / 2) * torch.norm((param - global_weights[p_i])) ** 2
        loss += prox_term

        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        losses.append(loss.item())  # 将每次训练的损失值添加到列表中
    return losses

def train_model_FedNova_local(input_model,X_train_tensor, y_train_tensor, num_epochs):
    # because last round trained global model replaced local model,
    # that in this round the first local model is last round global model
    losses =[]
    model = input_model
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    global_weights = copy.deepcopy(input_model.state_dict())
    tau = 0
    rho = 0.9
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        tau +=len(y_train_tensor)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        losses.append(loss.item())
    coeff = (tau - rho * (1 - pow(rho, tau)) / (1 - rho)) / (1 - rho)
    state_dict = model.state_dict()
    norm_grad = copy.deepcopy(global_weights)
    for key in norm_grad:
        norm_grad[key] = torch.div(global_weights[key] - state_dict[key], coeff)

    return losses, coeff, norm_grad,len(X_train_tensor)
# 定义模型参数共享函数
def share_params(model):
    params = model.state_dict()
    # return {k: v.clone().detach().requires_grad_(True) for k, v in params.items()}
    return {k: v.clone().detach().requires_grad_(False) for k, v in params.items()}

# # 定义模型参数聚合函数
def aggregate_params(params_list):
    aggregated_params = {}
    for key in params_list[0].keys():
        # 将参数转换为张量进行处理
        params_tensors = [params[key].clone().detach().float() for params in params_list]
        # 聚合参数
        aggregated_params[key] = sum(params_tensors) / len(params_tensors)
    return aggregated_params
def test(global_model,X_test,y_test):
    # 在全局模型上进行测试
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test)
        y_test_tensor = torch.LongTensor(y_test)
        outputs = global_model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
        print(f"Test Accuracy: {accuracy * 100:.2f}%")

        # 计算度量
        predictions = predicted.numpy() # 将张量转换为 NumPy 数组并去除零维数组
        true_labels = y_test_tensor.numpy()  # 将张量转换为 NumPy 数组并去除零维数组
        precision = precision_score(true_labels,predictions,zero_division=0.0,average=None)
        precision_micro = precision_score(true_labels,predictions,zero_division=0.0,average='micro')
        precision_macro = precision_score(true_labels,predictions,zero_division=0.0,average='macro')
        # recall
        recalls = recall_score(true_labels,predictions,zero_division=0.0,average=None)
        recalls_micro =recall_score(true_labels,predictions,zero_division=0.0,average='micro')
        recalls_macro =recall_score(true_labels,predictions,zero_division=0.0,average='macro')
        f1_scores = f1_score(true_labels, predictions, average=None)
        acc = accuracy_score(true_labels, predictions)
        kappa = cohen_kappa_score(true_labels,predictions)
        conf_matrix = confusion_matrix(true_labels,predictions)
        # 计算所有类别乘积的几何平均值作为 G-mean
        g_mean_all= np.power(np.prod(recalls), 1 / len(recalls))
        # AUC
        lb = LabelBinarizer()
        lb.fit(true_labels)
        true_labels_bin = lb.transform(true_labels)
        predictions_bin = lb.transform(predictions)
        auc = roc_auc_score(true_labels_bin, predictions_bin, average='weighted', multi_class='ovr')
        metrics = {
            'recall':recalls,
            'recall_micro':recalls_micro,
            'recall_macro':recalls_macro,
            'precision':precision,
            'precision_micro':precision_micro,
            'precision_macro':precision_macro,
            'f1_score':f1_scores,
            'g_mean':g_mean_all,
            'acc':acc,
            'auc':auc,
            'kappa':kappa,
            'confusion_matrix':conf_matrix
        }
        return metrics

def save_loss(loss_list,client_id,round_id,save_loss_path):
    if not os.path.exists(save_loss_path):
        os.makedirs(save_loss_path)
    # 构建文件路径
    file_path = os.path.join(save_loss_path, f"client_{client_id}.csv")

    if os.path.exists(file_path):
        # 如果文件存在，加载现有的 CSV 文件为 DataFrame
        df = pd.read_csv(file_path)
    else:
        # 如果文件不存在，直接创建新的 DataFrame
        df = pd.DataFrame()

    # 将损失值添加到 DataFrame 中
    column_name = f'round_{round_id}'
    df[column_name] = loss_list

    # 将 DataFrame 保存为 CSV 文件
    df.to_csv(file_path, index=False)
def save_model(global_model,round_id,save_model_path):
    if not os.path.exists(save_model_path):
        os.makedirs(save_model_path)
    model_path = os.path.join(save_model_path,f'round_{round_id}_gm.pt')
    torch.save(global_model,model_path)

def save_metrics(title, rounds, metrics, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    file_name = f"{title}.csv"
    file_path = os.path.join(save_folder, file_name)
    recalls = metrics['recall']
    class_nums = len(recalls)
    # 检查文件是否存在
    if os.path.exists(file_path):
        # 如果文件存在，加载现有的 Excel 文件为 DataFrame
        df = pd.read_csv(file_path)
    else:
        # 如果文件不存在，直接创建新的 DataFrame
        columns = [
        'rounds', 'accuracy', 'auc', 'kappa', 'g_mean', 'recall_micro', 'precision_micro',
        'recall_macro', 'precision_macro'
        ]
        df = pd.DataFrame(columns=columns)
        for i in range(class_nums):  # 动态生成 f1-score 相关列名
            columns.append(f'f1_score_{i}')
            columns.append(f'recall_{i}')
            columns.append(f'precession_{i}')

    data = {
        'rounds': rounds,
        'accuracy': metrics['acc'],
        'auc': metrics['auc'],
        'kappa': metrics['kappa'],
        'g_mean':metrics['g_mean'],
        'recall_micro':metrics['recall_micro'],
        'precision_micro':metrics['precision_micro'],
        'recall_macro':metrics['recall_macro'],
        'precision_macro':metrics['precision_macro']
    }
    # 添加每个类别的 F1-score、G-mean 和 Recall 到 data 中
    for i in range(class_nums):  #类别数
        data[f'recall_{i}'] = metrics['recall'][i]
        data[f'precision_{i}'] = metrics['precision'][i]
        data[f'f1_score_{i}'] = metrics['f1_score'][i]
    # 创建新行并追加到 DataFrame
    new_row = pd.DataFrame(data, index=[0])
    df = pd.concat([df, new_row], ignore_index=True)

    # 将 DataFrame 保存为 Excel 文件
    df.to_csv(file_path, index=False)

def inremental_sampling(prototype_map,raw_X,raw_y,cluster_strategy):
    class_nums = len(np.unique(raw_y))
    print("class nums" ,class_nums)
    isap = Incremental_sampling2()
    # resampling_data,resampling_label,prototype_map = isap.fit(last_round_prototype=prototype_map,data=raw_X,label=raw_y)
    # def fit(self,new_data,new_data_label,last_round_prototype,sampling_strategy='tpl'):
    def fit(self,new_data,new_data_label,last_round_prototype,sampling_strategy='tpl'):
        self.incremental_prototypes = last_round_prototype
        self.data = new_data
        self.label = new_data_label
        self.data_combined()
        resampling_data = []
        resampling_label = []
        min_length, max_length =self.compute_sampling_nums(self.incremental_prototypes)
        if sampling_strategy.lower() == "tpl":
            resampling_data,resampling_label = self.triplet_sampling(max_length)
        elif sampling_strategy.lower() == "ros-p":
            # rest data copied from prototype
            resampling_data,resampling_label = self.random_sampling(max_length,sampling_strategy = "ros-p")
        elif sampling_strategy.lower() == "ros-h":
            # rest data copied from hybrid data(combined data)
            resampling_data,resampling_label = self.random_sampling(max_length,sampling_strategy = "ros-h")
        return resampling_data,resampling_label,self.incremental_prototypes
    resampling_data,resampling_label,prototype_map = isap.fit(new_data=raw_X,new_data_label=raw_y,last_round_prototype=prototype_map,sampling_strategy=cluster_strategy)
    return resampling_data,resampling_label,prototype_map
def read_data_return_tensor(dataset_path, round_id, client_id, sampling_strategy='no',prototype_map = {},cluster_strategy='kmeans'):
    folder_path = os.path.join(dataset_path, f'client_{client_id}')
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    open_file_path = os.path.join(folder_path, f'round_{round_id}.csv')
    data = pd.read_csv(open_file_path, header=0)
    # 排除标题行并打乱数据
    data_shuffled = shuffle(data)  # 排除标题行并打乱数据

    # 提取特征和目标变量
    raw_X = data_shuffled.iloc[:, :-1].values  # 特征
    raw_y = data_shuffled.iloc[:, -1].values   # 目标变量
    raw_y = raw_y.astype(float)
    # print(len(raw_X),type(raw_X))
    # print(len(raw_X),type(raw_X))
    if sampling_strategy == 'IncrementalSampling2':
        print("using IncrementalSampling")
        resampling_X,resampling_y,prototype_map = inremental_sampling(prototype_map,raw_X,raw_y,cluster_strategy=cluster_strategy)
    # else:
    #     resampling_X,resampling_y = data_sampling(raw_X,raw_y,sampling_strategy)
    #     resampling_y.astype(float)
    resampling_X = np.array(resampling_X)  # 将列表转换为单个NumPy数组
    resampling_y = np.array(resampling_y)  # 将列表转换为单个NumPy数组

    X_train_tensor = torch.tensor(resampling_X, dtype=torch.float32)  # 特征张量
    y_train_tensor = torch.tensor(resampling_y, dtype=torch.long)   # 标签张量
    return X_train_tensor,y_train_tensor,prototype_map
def read_test_data(test_data_path):
    data = pd.read_csv(test_data_path, header=0)
    # 排除标题行并打乱数据
    data_shuffled = shuffle(data)  # 排除标题行并打乱数据

    # 提取特征和目标变量
    X = data_shuffled.iloc[:, :-1].values.astype(float)  # 特征
    # print(X)
    y = data_shuffled.iloc[:, -1].values   # 目标变量
    y = y.astype(float)

    X_test_tensor = torch.tensor(X, dtype=torch.float32)  # 特征张量
    y_test_tensor = torch.tensor(y, dtype=torch.long)   # 标签张量
    return X_test_tensor,y_test_tensor

In [3]:
def aggregate_fednova(local_params_list,gm):
    # (share_params(clients_models[i]),coeff, norm_grad, data_len) as input
    total_data_len = sum(data_len for _, _, _, data_len in local_params_list)
    global_model_state = gm.state_dict()
    nova_model_state = copy.deepcopy(global_model_state)
    # avg_loss = 0
    coeff = 0.0
    for clientID,(client_model,client_coeff,client_norm_grad,client_local_data_len) in enumerate(local_params_list):
        coeff = coeff + client_coeff*client_local_data_len/total_data_len
        for key in client_model.state_dict():
            if clientID == 0:
                nova_model_state[key] = client_norm_grad[key] * client_local_data_len/total_data_len
            else:
                nova_model_state[key] =nova_model_state[key]+ client_norm_grad[key] * client_local_data_len/total_data_len
        # avg_loss = avg_loss + cl
    for key in global_model_state:
        global_model_state[key] -= coeff*nova_model_state[key]

    return global_model_state
pp = []
def runFedNova(samplingName,settingName,cluster_strategy):
    num_clients = 10
    # 初始化全局模型和客户端模型
    input_size = 10
    hidden_size = 100
    output_size = 5
    global_model = MLP(input_size, hidden_size, output_size)
    client_prototype_map = [{} for _ in range(num_clients)]
    clients_models = [MLP(input_size, hidden_size, output_size) for _ in range(num_clients)]

    num_epochs = 200
    num_global_updates = 195
    # 'E:/FedStream/real_data_set/realdataset0427/elecNorm/Electricity_client_random/'
    base_path ='E:/FedStream/' # 'E:/FedStream/'
    dataset_name = 'pokerhand_five'
    setting_name = settingName # 'CovA_Abrupt_BtoVH'
    # SMOTE cant use in VH,H
    # sampling_strategy_name_list = ['no','RandomOverSampler','RandomUnderSampler','CondensedNearestNeighbour','Triplets','IncrementalSampling']
    # sampling_strategy_name = sampling_strategy_name_list[sampling_id]
    sampling_strategy_name = samplingName

    algorithm = 'FedNova_kvalue30'
    experiment_times = 'epoch200'
    save_loss_path = f'{base_path}/loss/{dataset_name}_Sampling/{algorithm}/{setting_name}/{sampling_strategy_name}_{cluster_strategy}kdt_{experiment_times}'
    save_model_path = f'{base_path}/models/{dataset_name}_Sampling/{algorithm}/{setting_name}/{sampling_strategy_name}_{cluster_strategy}kdt_{experiment_times}'
    save_metrics_path  = f'{base_path}/metrics/{dataset_name}_Sampling/{algorithm}/{setting_name}/{sampling_strategy_name}_{cluster_strategy}kdt_{experiment_times}'
    # E:\FedStream\real_data_set\realdataset0427\covertypeNorm\CoverType_client_Full_class
    read_data_path = f'E:/FedStream/real_data_set/realdataset0427/{dataset_name}/{setting_name}/'
    # 读取测试集CSV文件并转换为PyTorch张量
    # E:\FedStream\real_data_set\realdataset0427\covertypeNorm
    test_path = 'E:/FedStream/real_data_set/realdataset0427/pokerhand_five/test.csv'
    X_test_tensor,y_test_tensor = read_test_data(test_path)

    for update in range(num_global_updates):
        print(f"round{update}")
        local_params_list = []

        for i in range(num_clients):
            # 在每个客户端训练本地模型
            print(f"client{i}")
            X_train_local,y_train_local,prototype_map_r = read_data_return_tensor(read_data_path,round_id=update,client_id=i,sampling_strategy=sampling_strategy_name,prototype_map=client_prototype_map[i],cluster_strategy=cluster_strategy)
            client_prototype_map[i] = prototype_map_r
            # 训练本地模型并获取损失值
            losses, coeff, norm_grad, data_len = train_model_FedNova_local(clients_models[i],X_train_local,y_train_local,num_epochs=num_epochs)
            # y_train_tensor = torch.sub(y_train_tensor, 1)
            # losses, coeff, norm_grad, data_len = train_model_FedNova_local(clients_models[i],X_train_local,torch.sub(y_train_local, 1),num_epochs=num_epochs)
            # save_loss(loss_list=losses,client_id=i,round_id=update,save_loss_path=save_loss_path)

            local_metrics = test(copy.deepcopy(clients_models[i]),X_test_tensor,y_test_tensor)
            local_params_list.append((copy.deepcopy(clients_models[i]),coeff, norm_grad, data_len))
            save_metrics(title=f"client_{i}_metrics", rounds=update, metrics=local_metrics,save_folder = save_metrics_path)

        aggregated_params = aggregate_fednova(local_params_list,gm = copy.deepcopy(global_model))
        global_model.load_state_dict(aggregated_params)

        # 在每轮结束后发送全局模型参数给客户端
        gm = copy.deepcopy(global_model)
        save_model(copy.deepcopy(gm),update,save_model_path)
        for client_model in clients_models:
            client_model.load_state_dict(gm.state_dict())

        me = test(gm,X_test_tensor,y_test_tensor) # torch.sub(y_train_tensor, 1)
        # me = test(gm,X_test_tensor,torch.sub(y_test_tensor, 1))
        save_metrics(title="global_back", rounds=update, metrics=me,save_folder = save_metrics_path)
        print("gme acc:" ,me)
import time

sampling_strategy_name_list =['IncrementalSampling'] # ['no','IncrementalSampling','RandomOverSampler', 'RandomUnderSampler', 'CondensedNearestNeighbour', 'Triplets']#, 'RandomOverSampler', 'RandomUnderSampler', 'CondensedNearestNeighbour', 'Triplets', 'IncrementalSampling']
dataset_list = ['pokerhand_client']#,'pokerhand_client','pokerhand_client','pokerhand_client','pokerhand_client']#,'CoverType_client_Full_class','CoverType_client_Full_class','CoverType_client_Full_class','CoverType_client_Full_class']
cluster_strategy_list = ['ros-h'] # 'ros-p','ros-p']# ['gmm','kmeans++','OPTICS','meanshift'] # 'kmeans','kmeans++','OPTICS','meanshift','spectral','kmedoids',
total_start_time = time.time()  # 记录整个循环的开始时间
ex_time_list = []
for j, settingname in enumerate(dataset_list):
    for i, cluster_strategy in enumerate(cluster_strategy_list):
        start_time = time.time()  # 记录当前迭代的开始时间
        runFedNova(samplingName='IncrementalSampling2', settingName=settingname,cluster_strategy=cluster_strategy)
        end_time = time.time()  # 记录当前迭代的结束时间
        execution_time = end_time - start_time  # 计算当前迭代的执行时间
        ex_time_list.append(execution_time)
        time.sleep(20)

total_end_time = time.time()  # 记录整个循环的结束时间
total_execution_time = total_end_time - total_start_time  # 计算整个循环的执行时间
# kdt ros-p(11hours)
print(f"Total execution time: {total_execution_time} seconds")
print(ex_time_list)
# [34400.61084651947, 36640.83967113495]
# Total execution time: 69354.06012558937 seconds
# [33072.67344403267, 36241.37223672867]
# import time
#
# sampling_strategy_name_list = ['IncrementalSampling2']#, 'RandomOverSampler', 'RandomUnderSampler', 'CondensedNearestNeighbour', 'Triplets', 'IncrementalSampling']
# dataset_list = ['pokerhand_client']#,'CoverType_client_Full_class','CoverType_client_Full_class','CoverType_client_Full_class','CoverType_client_Full_class']
#
# total_start_time = time.time()  # 记录整个循环的开始时间
# ex_time_list = []
# for j, settingname in enumerate(dataset_list):
#     for i, saplingname in enumerate(sampling_strategy_name_list):
#         start_time = time.time()  # 记录当前迭代的开始时间
#         runFedNova(samplingName=saplingname, settingName=settingname,cluster_strategy='is2_triplets')
#         end_time = time.time()  # 记录当前迭代的结束时间
#         execution_time = end_time - start_time  # 计算当前迭代的执行时间
#         ex_time_list.append(execution_time)
#         time.sleep(20)
#
# total_end_time = time.time()  # 记录整个循环的结束时间
# total_execution_time = total_end_time - total_start_time  # 计算整个循环的执行时间
#
# print(f"Total execution time: {total_execution_time} seconds")
# print(ex_time_list)
# Total execution time: 69354.06012558937 seconds
# [33072.67344403267, 36241.37223672867]
# Total execution time: 36974.078832149506 seconds
# [36954.06793355942]
# 没有电风扇冷却
# Total execution time: 47523.148163080215 seconds
# [47503.13124418259]

Epoch [141/200], Loss: 0.0731
Epoch [151/200], Loss: 0.0731
Epoch [161/200], Loss: 0.0730
Epoch [171/200], Loss: 0.0729
Epoch [181/200], Loss: 0.0728
Epoch [191/200], Loss: 0.0727
Test Accuracy: 97.13%
client9
using IncrementalSampling
class nums 5
use ros h
Epoch [1/200], Loss: 0.0780
Epoch [11/200], Loss: 0.0782
Epoch [21/200], Loss: 0.0774
Epoch [31/200], Loss: 0.0776
Epoch [41/200], Loss: 0.0776
Epoch [51/200], Loss: 0.0775
Epoch [61/200], Loss: 0.0770
Epoch [71/200], Loss: 0.0774
Epoch [81/200], Loss: 0.0769
Epoch [91/200], Loss: 0.0769
Epoch [101/200], Loss: 0.0768
Epoch [111/200], Loss: 0.0768
Epoch [121/200], Loss: 0.0764
Epoch [131/200], Loss: 0.0767
Epoch [141/200], Loss: 0.0764
Epoch [151/200], Loss: 0.0764
Epoch [161/200], Loss: 0.0762
Epoch [171/200], Loss: 0.0762
Epoch [181/200], Loss: 0.0760
Epoch [191/200], Loss: 0.0759
Test Accuracy: 96.38%
Test Accuracy: 96.58%
gme acc: {'recall': array([0.93842803, 0.99218103, 1.        , 1.        , 0.9979571 ]), 'recall_micro': 0.9

In [1]:
print("a")

a


In [2]:
print('v;')

v;
